In [2]:
#import standard libs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from random import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Conv1D, LSTM, Flatten

In [3]:
#import project libs
from PowerAIDataHandler.PowerAIDataHandler import ClassPowerAIDataHandler

#corrupt timestamps: Time frame: 1728835915654/1728845708532/
#  Sprung zwischen 1728835995276 und 1728845640410

In [4]:
#init datahandler using configuration from .env
dh = ClassPowerAIDataHandler("C:/Users/Kids/iCloudDrive/Documents/GitHub/PowerAI/.env")

#init parameters for neural network
num_devices   = len(dh.device_list)   #länge der Gerätliste
event_ratio   = 0.5 #Verhältnis Trainings-/Testdaten
window_length = 15 #Anzahl an Daten die gleichzeitig eingespeist werden
epochs        = 100 #Anzahl der Traings-/Testdurchläufe

In [5]:
#read the calssified events from the mysql database
dh.read_events_from_db()

#print timeserie of the events
dh.print_events()

URLError: <urlopen error [WinError 10060] Ein Verbindungsversuch ist fehlgeschlagen, da die Gegenstelle nach einer bestimmten Zeitspanne nicht richtig reagiert hat, oder die hergestellte Verbindung war fehlerhaft, da der verbundene Host nicht reagiert hat>

In [ ]:
#dh.filter_events_by_minpow()
#dh.print_events()

In [ ]:
x = pd.to_datetime(dh.event_list[32][0]['timestamp'], utc=True, unit='ms')
y = np.array(dh.event_list[32][0]['value']).astype(float)


x = x[0:240]
y = y[0:240]

f = plt.figure()
a = f.add_subplot()
a.plot(x, y, marker = 'o')

In [ ]:
#test_x = { 'timestamp' : np.array([]), 'value' : np.array([]), 'device' : np.array([]) }
#for key in dh.device_list.keys():
 #   for i in range(len(dh.event_list[key])):
  #      test_x['timestamp'] = np.append(test_x['timestamp'], dh.event_list[key][i]['timestamp'])
   #     test_x['value'] = np.append(test_x['value'], dh.event_list[key][i]['value'])
    #    test_x['device'] = np.append(test_x['device'], dh.event_list[key][i]['device'])

In [8]:
test_xx = { 'timestamp' : np.array([]), 'value' : np.array([]), 'device' : np.array([]) }
for key in dh.device_list.keys():
    for i in range(len(dh.event_list[key])):
        test_xx['timestamp'] = np.append(test_xx['timestamp'], dh.event_list[key][i]['timestamp'])
        test_xx['value'] = np.append(test_xx['value'], dh.event_list[key][i]['value'])
        test_xx['device'] = np.append(test_xx['device'], dh.event_list[key][i]['device'])

In [9]:
print(len(test_xx['value']))

0


In [10]:
test_x = { 'timestamp' : np.array([]), 'value' : np.array([]), 'device' : np.array([]) }
for i in range(len(test_xx['value'])):
    if test_xx['device'][i] == 1:
        test_x['timestamp'] = np.append(test_x['timestamp'], test_xx['timestamp'][i])
        test_x['value'] = np.append(test_x['value'], test_xx['value'][i])
        test_x['device'] = np.append(test_x['device'], test_xx['device'][i])

In [11]:
print(len(test_x['value']))

0


In [ ]:
   # if test_xx['device'][i] == 2:
    #    test_x['timestamp'] = np.append(test_x['timestamp'], test_xx['timestamp'][i])
     #   test_x['value'] = np.append(test_x['value'], test_xx['value'][i])
      #  test_x['device'] = np.append(test_x['device'], test_xx['device'][i])
    #if test_xx['device'][i] == 64:
     #   test_x['timestamp'] = np.append(test_x['timestamp'], test_xx['timestamp'][i])
      #  test_x['value'] = np.append(test_x['value'], test_xx['value'][i])
       # test_x['device'] = np.append(test_x['device'], test_xx['device'][i])
        
        

In [ ]:
#Programm zur Erkennung von espresso-machine
y = np.zeros(len(test_x['value']))
espresso = 1
start_espresso = False
for i in range(len(test_x['value']) - 1):
    if test_x['value'][i+1] - test_x['value'][i] > 950 and test_x['value'][i+1] - test_x['value'][i] < 1050:
        y[i+1] = espresso
        start_espresso = True
    if test_x['value'][i+1] - test_x['value'][i] > -850 and test_x['value'][i+1] - test_x['value'][i] < -950:
        y[i] = espresso
        start_espresso = False
    if start_espresso == True:
        y[i] = espresso

In [ ]:
#Programm zur Erkennung von microwave
microwave = 64
start_microwave = False
for i in range(len(test_x['value']) - 3):
    if test_x['value'][i+3] - test_x['value'][i] > 1150 and test_x['value'][i+3] - test_x['value'][i] < 1250:
        y[i+3]= microwave
        start_microwave = True
    if test_x['value'][i+3]- test_x['value'][i] > -1150 and test_x['value'][i+3] - test_x['value'][i] < -1250:
        y[i] = microwave
        start_microwave = False
    if start_microwave == True:
        y[i] = microwave

In [ ]:
#Programm zur Erkennung von kitchen-light
kitchen = 128
start_kitchen = False
for i in range(len(test_x['value']) - 2):  
    if test_x['value'][i+2] - test_x['value'][i] > 270 and test_x['value'][i+2] - test_x['value'][i] < 310:
        y[i+2] = kitchen
        start_kitchen = True
    if test_x['value'][i+2] - test_x['value'][i] > -270 and test_x['value'][i+2] - test_x['value'][i] < -310:
        y[i] = kitchen
        start_kitchen = False
    if start_kitchen == True:
        y[i] = kitchen

In [ ]:
#Programm zur Erkennung von washing-machine
washing = 2
start_washing = False
for i in range(len(test_x['value'])-150):
    if test_x['value'][i+150] - test_x['value'][i] > 2150 and test_x['value'][i+150] - test_x['value'][i] < 2300:
        y[i+15] = washing
        start_washing = True
        start_i = i
    if start_washing == True and i - start_i >= 7200:
        start_washing = False
    if start_washing == True:
        y[i] = washing

In [ ]:
#Programm zur Erkennung von dish-washer
dish = 4
start_dish = False
for i in range(len(test_x['value'])-6):
    if test_x['value'][i+6] - test_x['value'][i] > 2200 and test_x['value'][i+6] - test_x['value'][i] < 2300:
        y[i+6] = dish
        start_dish = True
        start_i = i
    if start_dish == True and i - start_i >= 6300:
        start_dish = False
    if start_dish == True:
        y[i] = dish

In [ ]:
#Programm zur Erkennung von induction-cooker
y = np.zeros(len(test_x['value']))
induction = 8
start_induction = False
for i in range(len(test_x['value']) - 600):
    if test_x['value'][i+1] - test_x['value'][i] > 950 and test_x['value'][i+1] - test_x['value'][i] < 1050:
        y[i+1] = induction
        start_induction = True
    if np.var(test_x['value'][i:i+600]) < 430000:
        start_induction = False
    if start_induction == True:
        y[i] = induction

In [ ]:
#Programm zur Erkennung von oven
y = np.zeros(len(test_x['value']))
oven = 32
start_oven = False
for i in range(len(test_x['value']) - 600):
    if test_x['value'][i+1] - test_x['value'][i] > 950 and test_x['value'][i+1] - test_x['value'][i] < 1050:
        y[i+1] = oven
        start_oven = True
    if np.var(test_x['value'][i:i+600]) < 430000:
        start_oven = False
    if start_oven == True:
        y[i] = oven

In [12]:
for i in range(len(dh.event_list[8])):
    print(f"Varianz von Ofen-Event= {np.var(dh.event_list[8][i]['value'])}")        

In [ ]:
#create vector of data
#data = test_x['value'] if test_x['timestamp']
#x = pd.to_datetime(dh.event_list[1][26]['timestamp'], utc=True, unit='ms')
#y = np.array(dh.event_list[1][26]['value']).astype(float)
#define function to calculate cv
#cv = lambda x: np. std (x, ddof= 1 ) / np. mean (x) * 100 

#calculate CV
#cv(data)

In [ ]:
 #Programm zur Erkennung von light

living = 256
start_living = False
for i in range(len(test_x['value']) - 2):
    if test_x['value'][i+1] - test_x['value'][i] > 185 and test_x['value'][i+1] - test_x['value'][i] < 195:
        y[i+1] = living
        start_living = True
    if test_x['value'][i+1] - test_x['value'][i] > -185 and test_x['value'][i+1] - test_x['value'][i] < -195:
        y[i] = living
        start_living = False
    if start_living == True:
        y[i] = living

In [ ]:
#Programm zur Erkennung von light

dining = 512
start_dining = False

for i in range(len(test_x['value']) - 1):
    if test_x['value'][i+1] - test_x['value'][i] > 25 and test_x['value'][i+1] - test_x['value'][i] < 35:
        y[i+1] = dining
        start_dining = True
    if test_x['value'][i+1] - test_x['value'][i] > -25 and test_x['value'][i+1] - test_x['value'][i] < -35:
        y[i] = dining
        start_dining = False
    if start_dining == True:
        y[i] = dining

In [ ]:
#Programm zur Erkennung von light

kitchen = 128
living = 256
dining = 512
ground = 1024
upper = 2048

start_kitchen = False
start_living = False
start_dining = False
start_ground = False
start_upper = False
for i in range(len(test_x['value']) - 2):
    
    if test_x['value'][i+2] - test_x['value'][i] > 270 and test_x['value'][i+2] - test_x['value'][i] < 315:
        y[i+2] = kitchen
        start_kitchen = True
    if test_x['value'][i+2] - test_x['value'][i] > -270 and test_x['value'][i+2] - test_x['value'][i] < -310:
        y[i] = kitchen
        start_kitchen = False
    if start_kitchen == True:
        y[i] = kitchen
 '''   
    if test_x['value'][i+1] - test_x['value'][i] > 185 and test_x['value'][i+1] - test_x['value'][i] < 195:
        y[i+1] = living
        start_living = True
    if test_x['value'][i+1] - test_x['value'][i] > -185 and test_x['value'][i+1] - test_x['value'][i] < -195:
        y[i] = living
        start_living = False
    if start_living == True:
        y[i] = living
    
    if test_x['value'][i+1] - test_x['value'][i] > 25 and test_x['value'][i+1] - test_x['value'][i] < 35:
        y[i+1] = dining
        start_dining = True
    if test_x['value'][i+1] - test_x['value'][i] > -25 and test_x['value'][i+1] - test_x['value'][i] < -35:
        y[i] = dining
        start_dining = False
    if start_dining == True:
        y[i] = dining
        
    if test_x['value'][i+1] - test_x['value'][i] > 200 and test_x['value'][i+1] - test_x['value'][i] < 210:
        y[i+1] = ground
        start_ground = True
    if test_x['value'][i+1] - test_x['value'][i] > -200 and test_x['value'][i+1] - test_x['value'][i] < -210:
        y[i] = ground
        start_ground = False
    if start_ground == True:
        y[i] = ground
        
    if test_x['value'][i+1] - test_x['value'][i] > 130 and test_x['value'][i+1] - test_x['value'][i] < 150:
        y[i+1] = upper
        start_upper = True
    if test_x['value'][i+1] - test_x['value'][i] > -130 and test_x['value'][i+1] - test_x['value'][i] < -150:
        y[i] = upper
        start_upper = False
    if start_upper == True:
        y[i] = upper
'''        

In [ ]:
#for i in range(len(test_x['value'])):
  #  y[i] = test_x['device'][i]

In [ ]:
#Ergebnis vergleichen

correct = 0
total = len(test_x['value'])
for i in range(total):
    if test_x['device'][i] == y[i]:
        correct = correct + 1

print(f"Die Erfolgsquote beträgt {correct/total*100}%, insgesamt wurden {correct} Messpunkte von {total} korrekt zugeordnet.")

In [ ]:
#Ergebnis vergleichen
false = 0
total = len(test_x['value'])
for i in range(total):
    if test_x['device'][i] != y[i]:
        false = false + 1

print(f"Die Fehlerquote beträgt {false/total}, insgesamt wurden {false} Messpunkte von {total} falsch zugeordnet.")

In [ ]:
model = Sequential()
# input layer
model.add(Input(shape=(window_length)))
model.add(tf.keras.layers.Reshape(target_shape=(window_length, 1)))
# conv1 layer
model.add(tf.keras.layers.Convolution1D(filters=64, kernel_size=3, padding="same"))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.ReLU())
# conv2 layer
model.add(tf.keras.layers.Convolution1D(filters=64, kernel_size=3, padding="same"))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.ReLU())
# conv3 layer
model.add(tf.keras.layers.Convolution1D(filters=64, kernel_size=3, padding="same"))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.ReLU())
# gap
model.add(tf.keras.layers.GlobalAveragePooling1D())
# output layer
model.add(tf.keras.layers.Dense(num_devices, activation="softmax"))

In [ ]:
# Kompilieren des Modells
model.compile(optimizer='adam', loss='mse', metrics=['mse', 'msle', 'mae'])
# Anzeigen der Modellzusammenfassung
model.summary()

In [ ]:
#xx, yy = dh.generate_training_data_from_db(window_length)
train_x, train_y, test_x, test_y = dh.generate_training_data_from_events(window_length, event_ratio) #fk: aufteilen der Daten

In [ ]:
# train the model
history = model.fit(
            x=train_x,
            y=train_y,
            batch_size=train_x[0].size,
            epochs=epochs,
            verbose=0)

In [ ]:
plt.plot(history.history['loss'])


In [ ]:
#Prediction with neural network

predict_y = model.predict(test_x)
    
dh.compare_with_testdata(predict_y, test_x, test_y)

In [ ]:
# fixed potential bug in line 218. train data ratio: 0.5, with event filter
#+--------------------+-------+---------+-------+---------+
#|        device name | total | correct | wrong | percent |
#+--------------------+-------+---------+-------+---------+
#|   espresso-machine |   657 |     618 |    39 |     94% |
#|    washing-machine |   474 |       0 |   474 |      0% |
#|        dish-washer |   383 |     382 |     1 |     99% |
#|   induction-cooker |    73 |       0 |    73 |      0% |
#|  irrigation-system |     0 |       0 |     0 |         |
#|               oven |    18 |       7 |    11 |     38% |
#|          microwave |     0 |       0 |     0 |         |
#|      kitchen-light |    21 |       0 |    21 |      0% |
#|  living-room-light |     0 |       0 |     0 |         |
#|  dining-room-light |     0 |       0 |     0 |         |
#| ground-floor-light |    10 |       0 |    10 |      0% |
#|  upper-floor-light |     0 |       0 |     0 |         |
#+--------------------+-------+---------+-------+---------+